In [2]:
import ccxt
import time

# API Keys (Replace with actual keys)
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'

# Connect to exchanges
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

In [ ]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.5 / 100  # 0.5% profit target
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
TRADING_FEE = 0.001  # 0.1% trading fee per trade

def get_tradable_tokens(exchange, exchange_name, futures=False):
    """Fetches all tradable tokens on the given exchange and filters out low-volume pairs"""
    try:
        markets = exchange.load_markets()
        tradable_tokens = {"spot": [], "futures": []}

        for symbol in markets:
            # Identify if it's a Spot or Futures pair
            if futures:
                if ":USDT" in symbol:  # Futures trading pairs have a colon (Bybit & Bitget)
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0) or market_data.get('baseVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens["futures"].append(symbol)
            else:
                if "/USDT" in symbol and ":" not in symbol:  # Spot trading pairs
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0) or market_data.get('baseVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens["spot"].append(symbol)

        print(f"✅ Found {len(tradable_tokens['spot'])} spot tokens and {len(tradable_tokens['futures'])} futures tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return {"spot": [], "futures": []}

def get_price(exchange, symbol):
    """Fetch latest price for a given token if the market exists"""
    try:
        if symbol in exchange.load_markets():
            ticker = exchange.fetch_ticker(symbol)
            return float(ticker.get("last", ticker.get("lastPrice", None)))
        return None
    except Exception as e:
        print(f"❌ Error fetching price for {symbol}: {e}")
        return None

def check_inter_exchange_arbitrage(symbol_spot, symbol_futures):
    """Checks for arbitrage opportunities between Bybit and Bitget across Spot and Futures"""
    bybit_spot_price = get_price(bybit, symbol_spot)
    bitget_spot_price = get_price(bitget, symbol_spot)
    bybit_futures_price = get_price(bybit, symbol_futures)
    bitget_futures_price = get_price(bitget, symbol_futures)

    # Store possible trade options
    trade_options = []

    if bybit_spot_price and bitget_spot_price:
        trade_options.append(("Spot", "Spot", bybit_spot_price, bitget_spot_price))

    if bybit_spot_price and bitget_futures_price:
        trade_options.append(("Spot", "Futures", bybit_spot_price, bitget_futures_price))

    if bybit_futures_price and bitget_spot_price:
        trade_options.append(("Futures", "Spot", bybit_futures_price, bitget_spot_price))

    if bybit_futures_price and bitget_futures_price:
        trade_options.append(("Futures", "Futures", bybit_futures_price, bitget_futures_price))

    for buy_market, sell_market, buy_price, sell_price in trade_options:
        spread = (sell_price - buy_price) / buy_price
        net_profit = spread - (TRADING_FEE * 2)

        if net_profit >= SPREAD_THRESHOLD:
            print(f"🔍 Inter-exchange Arbitrage Detected: Buy on Bybit {buy_market}, Sell on Bitget {sell_market} for {symbol_spot}")
            print(f"    ✅ Bybit {buy_market} Price: {buy_price:.2f} USDT")
            print(f"    ✅ Bitget {sell_market} Price: {sell_price:.2f} USDT")
            print(f"    ✅ Spread: {spread:.4%}")
            print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
            print("-" * 50)

while True:
    try:
        bybit_tokens = get_tradable_tokens(bybit, "Bybit")
        bitget_tokens = get_tradable_tokens(bitget, "Bitget")

        common_spot_tokens = list(set(bybit_tokens["spot"]) & set(bitget_tokens["spot"]))
        common_futures_tokens = list(set(bybit_tokens["futures"]) & set(bitget_tokens["futures"]))

        if not common_spot_tokens and not common_futures_tokens:
            print("❌ No common tradable tokens found between Bybit and Bitget. Retrying in 10 seconds...")
            time.sleep(10)
            continue

        print("🔄 Checking for arbitrage opportunities...")

        for token in common_spot_tokens:
            futures_token = token.replace("/USDT", ":USDT")
            check_inter_exchange_arbitrage(token, futures_token)

        print("✅ Completed this cycle. Waiting for next check...")
        time.sleep(CHECK_INTERVAL)

    except Exception as e:
        print(f"❌ Error in main loop: {e}")
        time.sleep(10)

❌ Error fetching tradable tokens from Bybit: bybit GET https://api.bybit.com/v5/asset/coin/query-info?
❌ Error fetching tradable tokens from Bitget: bitget GET https://api.bitget.com/api/v2/spot/public/coins
❌ No common tradable tokens found between Bybit and Bitget. Retrying in 10 seconds...
❌ Error fetching tradable tokens from Bybit: bybit GET https://api.bybit.com/v5/asset/coin/query-info?
❌ Error fetching tradable tokens from Bitget: bitget GET https://api.bitget.com/api/v2/spot/public/coins
❌ No common tradable tokens found between Bybit and Bitget. Retrying in 10 seconds...
❌ Error fetching tradable tokens from Bybit: bybit GET https://api.bybit.com/v5/asset/coin/query-info?
❌ Error fetching tradable tokens from Bitget: bitget GET https://api.bitget.com/api/v2/spot/public/coins
❌ No common tradable tokens found between Bybit and Bitget. Retrying in 10 seconds...
❌ Error fetching tradable tokens from Bybit: bybit GET https://api.bybit.com/v5/asset/coin/query-info?
❌ Error fetchin